<a href="https://colab.research.google.com/github/ARAN1218/piedpiper-python/blob/main/PPP%E2%91%A4_Sazae_janken_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# サザエさんの手を予測して、じゃんけんで圧勝しよう！
Python講座の終着点、人工知能編へようこそ！  
Pythonはできることが様々あるのですが、やはり特筆すべきはデータ分析・機械学習のしやすさでしょう！  
ここではPythonの使い道の中で最も面白い機械学習をジャンケンAIを作ることで体験し、その沼へ進んでいきましょう！

皆さんはサザエさんを知っていますか？サザエさんは西暦１９６９年（昭和４４年）１０月５日（日曜日）から今までずっと放送されている国民的アニメーションですが、西暦１９９１年（平成３年）の秋に突然じゃんけんが行われたことはあまり知られていないはずです。小さい頃皆さんはこのじゃんけんに気軽に挑戦していたことと思われますが、これに対して初回からデータを取り続け、それを分析することで驚異の勝率8割越えを達成しているヤバい団体があります。つまり、我々も努力次第でサザエさんに圧勝できる可能性があるのです！！！  
今回は強敵・サザエさんに対し、じゃんけんAIを作ることで挑戦していきましょう！

とりあえずこのcolabをコピーして、マイドライブの「PiedPiper_Python_個人用」というフォルダの中に入れる作業をしましょう。そうしてから本編スタートです。

# 各種インポート

In [ ]:
import numpy as np
import pandas as pd
import requests
from bs4 import BeautifulSoup
from time import sleep
from tqdm.notebook import tqdm
import random
import pickle

from google.colab import drive
drive.mount('/content/drive')

# データ収集
敵を知り、己を知れば百戦危うからず...  
まずは敵を知ることが大切です。敵を知るには敵のデータが必要不可欠！  
**偶然にも**敵の全対戦データを保有している[サイト](http://park11.wakwak.com/~hkn/bunseki0.htm)があるらしい。しかし、その情報を一つ一つ手動でエクセルとかに入力するのは骨が折れる...  
そこで！そんな退屈なことはPythonにやってもらいましょ〜

## じゃんけんデータ（all）

In [ ]:
# http://park11.wakwak.com/~hkn/data{}.htm

# データを取得したいサイトのURLを文字列として保管する
# ここで、「/data{}.htm」と{}が入っている理由は、後でfstringのテクニックを使ってURLを変更したいからです！
url = "http://park11.wakwak.com/~hkn/data{}.htm"

# とりあえず1991年度のデータを取得したいので、fstringで目的のページにアクセスできるurlに書き換える
target_url = url.format(1991)

# target_urlのページにアクセスし、ページの文字情報等を変数rに入れる
r = requests.get(target_url)

# 連続でアクセスすると犯罪になる場合があるので、requests一回につき一秒以上待機時間をとる
# (今回は一回しかリクエストしないから本当はいらないけど)
sleep(1)

# rに入っているhtmlの文字情報を解析し、変数soupに入れる
soup = BeautifulSoup(r.content, 'html.parser')

# 取得したページ全体を表示してみましょう！
print(soup)

ヨシ！とりあえずサイトの情報を取得できましたね！  
ただ、このままだとやはり分析には使えませんよね...どうします？コピペでエクセルとかに入力しますか？  

違いますよね！退屈なことはPythonにやらせましょう！！

In [ ]:
data = soup.find('pre').text
data

テキスト中に「\n」が入ってて、汚いですよね。  
そんな時はそれを取り去って、かつそれを利用してデータを綺麗にリストに入れてやりましょう！

In [ ]:
data = data.strip("\n")
data = data.split('\n')
data

In [ ]:
data = [d.split(' ') for d in data]
data

In [ ]:
# pandas.DataFrame型のデータに変換する！
# データ分析には必須級の便利なやつです！
data = pd.DataFrame(data)
data

In [ ]:
# 1992以降もまとめてやりましょー

url = "http://park11.wakwak.com/~hkn/data{}.htm"
data_list = []

for year in tqdm(range(1991, 2023+1)): # tqdmを使うと、for文の進捗を可視化できる！(後かっこいい)
  target_url = url.format(year) # year年のデータにアクセスできるURLに変換する！
  r = requests.get(target_url)
  sleep(1)
  soup = BeautifulSoup(r.content, 'html.parser')
  data = soup.find('pre').text
  data = data.strip("\n")
  data = data.split('\n')
  data = [d.split(' ') for d in data]
  data_list.append(data) # 逐一データフレームに変換するのではなく、一旦リストに入れておく

df_all = pd.DataFrame(data_list) # 最後に一気に変換させる！
df_all.head() # データの先頭5行のみ表示させる

In [ ]:
# 試しに最後に取得した2023年度のデータを見てみると...？
data

作戦を変更する

In [ ]:
url = "http://park11.wakwak.com/~hkn/data{}.htm"
data_list = pd.DataFrame() # 保管用のデータフレーム

for year in tqdm(range(1991, 2023+1)): # tqdmを使うと、for文の進捗を可視化できる！(後かっこいい)
  target_url = url.format(year) # year年のデータにアクセスできるURLに変換する！
  r = requests.get(target_url)
  sleep(1)
  soup = BeautifulSoup(r.content, 'html.parser')
  data = soup.find('pre').text
  data = data.strip("\n")
  data = data.split('\n')
  data = [d.split(' ') for d in data]
  data = pd.DataFrame(data) # 今度は逐一データフレームに変換して、それを保管用のデータフレームに連結させる作戦でいく！
  data_list = pd.concat([data_list, data], axis=0)

df_all = pd.DataFrame(data_list)
df_all

NaNってNaNNaNだ？？  
NaNとは、存在しないデータに対してとりあえず割り当てられるハズレの値のことNaNです。今回のデータで言うと、第一回の行には存在しなかった列でも後ろの方の行には存在した列があり、その列のデータが無理やり作られてしまったことによりNaNが生じてしまったのですね。  
とりあえずこれをNaNとかしよう！...と考えるのはNaNセンスで、重要な情報が取得できているかもしれないので確認するのが無NaNでしょう。

In [ ]:
# 4列目と5列目に入っているデータの正体を見る
print(df_all[3].unique())
print()
print(df_all[4].unique())

列名(カラム名)が番号でダサい＆分かりにくいので、カラム名を変更しよう！  
この時、**カラム名を見ただけで列のデータの種類やデータ型が把握できる**とgoodですね。  
今回は以下のように命名しましょー

| 変更前 | 変更後 | 列の意味 |
| :-: | :-: | :-: |
| 0 | seq | 放送回数 |
| 1 | date | 日付 |
| 2 | hand | 宿敵・サザエさんが出したジャンケンの手 |
| 3 | detail1 | 補足情報① |
| 4 | detail2 | 補足情報② |

In [ ]:
# 列名を変更する
df_all = df_all.set_axis(['seq', 'date', 'hand', 'detail1', 'detail2'], axis=1) # axisは行：０、列：1で覚えましょう。もしくはUSBメモリみたいに2分の１を引きましょう。
df_all.head()

In [ ]:
# 収集したデータを保存してみよう！
# df_all.to_csv("drive/MyDrive/PiedPiper_Python_個人用/sazae_janken_raw.csv", index=False)

In [ ]:
# 保存したデータを読み込む！
df_all = pd.read_csv("drive/MyDrive/PiedPiper_Python_個人用/sazae_janken_raw.csv")
df_all

どうです？簡単にデータをゲットできたでしょ？  
ちなみにこの技術を「**スクレイピング**」と言います。英単語のscrape(こする)から来ています。

## じゃんけんデータ（describe）
サザエさんじゃんけん研究所のサイトには各年度の集計情報があるので、それも取得してしまいましょう！  
今度はさっきより断然簡単ですよ〜

In [ ]:
# pandasのread_htmlメソッドを使うことで、tableタグのデータをたった一行でスクレイピングできます！
# リストとして取得するので、その一つ目の要素を取り出す意味で[0]をつける
df_describe = pd.read_html('http://park11.wakwak.com/~hkn/bunseki0.htm')[0]
df_describe

In [ ]:
# これも保存しておく
#df_describe.to_csv("drive/MyDrive/PiedPiper_Python_個人用/sazae_describe_raw.csv", index=False)

In [ ]:
# 再度呼び出し
df_describe = pd.read_csv("drive/MyDrive/PiedPiper_Python_個人用/sazae_describe_raw.csv")
df_describe

# データ前処理
データが汚ったねぇから綺麗にする

## df_describe

In [ ]:
# まずはdf_describeから
columns = df_describe.iloc[0,:] # 一行目を列名にする為に取得する
columns[0] = "年" # NaNと入れ替え
df_describe = df_describe.set_axis(columns, axis=1)
df_describe = df_describe.iloc[1:,:] # 不要な行を取り除く
df_describe = df_describe.reset_index(drop=True)
df_describe

In [ ]:
# それぞれの手を回数と割合に分割する
# やってる操作：
#   1. .str.replace(")", "")...「)」の文字を消す
#   2. .str.split("(")...「(」の文字を軸にテキストを二分割する ex.) 18(34.6% → 18 & 34.6%
#   3.  .apply(pd.Series)...列を二つに分割する
#   4.  .set_axis(['〇〇(回数)', '〇〇(割合)'], axis=1)...列名を変更する
# こんな感じで一行にまとめて処理かけるけど、見ての通り可読性が落ちるから、チームで分析するときはやらないようにしよう！
gu = df_describe['グー'].str.replace(")", "").str.split("(").apply(pd.Series).set_axis(['グー(回数)', 'グー(割合)'], axis=1)
choki = df_describe['チョキ'].str.replace(")", "").str.split("(").apply(pd.Series).set_axis(['チョキ(回数)', 'チョキ(割合)'], axis=1)
pa = df_describe['パー'].str.replace(")", "").str.split("(").apply(pd.Series).set_axis(['パー(回数)', 'パー(割合)'], axis=1)

# それらを横方向に結合する
df_describe = pd.concat([df_describe, gu, choki, pa], axis=1)

# 分割前の不要な列を消す
df_describe = df_describe.drop(['グー', 'チョキ', 'パー'], axis=1)
df_describe

In [ ]:
# 不要な文字（「年」「%」）を一気に取り除く
df_describe['年'] = df_describe['年'].str.replace('年', '')
df_describe['グー(割合)'] = df_describe['グー(割合)'].str.replace('%', '')
df_describe['チョキ(割合)'] = df_describe['チョキ(割合)'].str.replace('%', '')
df_describe['パー(割合)'] = df_describe['パー(割合)'].str.replace('%', '')
df_describe

# 実は以下のようにまとめて処理もできたりする
# df_describe.applymap(lambda x : x.translate(str.maketrans({'年':'', '%':''})))

In [ ]:
# 最後にデータ型を数値に変える
# 変換前のデータ型
print(df_describe.dtypes)
print()

# astypeメソッドは辞書でデータ型を指定できる
astype_dict = {
    '年':str,
    '合計':int,
    'グー(回数)':int,
    'グー(割合)':float,
    'チョキ(回数)':int,
    'チョキ(割合)':float,
    'パー(回数)':int,
    'パー(割合)':float
}
df_describe = df_describe.astype(astype_dict)

# 変換後のデータ型
print(df_describe.dtypes)

In [ ]:
# 完成したデータを保存する
#df_describe.to_csv('drive/MyDrive/PiedPiper_Python_個人用/sazae_describe.csv', index=False)

## df_all

In [ ]:
# 次にdf_allをやる
# データが多いので、value_countsメソッドで各種値を確認
for column in df_all.columns:
  display(df_all[column].value_counts())

# handカラムにじゃんけんの手以外のデータが入っている→不要なデータの条件に使えそう

In [ ]:
# NaNが入っているかどうかを調べるには以下
for column in df_all.columns:
  print(column)
  print(sum(df_all[column].isna())) # isnaメソッドでTrue・Falseの一列にし、その内のTrue(=NaNの行)を合計することでNaNの個数が分かる
  print()

In [ ]:
# handカラムが「グー」「チョキ」「パー」以外であれば、その列を消す
df_all = df_all[df_all["hand"].isin(["グー", "チョキ", "パー"])]
df_all = df_all.reset_index(drop=True) # indexが飛び飛びになってしまっているので、リセットする
df_all

上の〇〇rowsのところを見てください。seqデータの最後の回数と同じになってますか？  
同じになってたら成功です！

次はdateのところをもっと便利に使える形に変換しましょう。

In [ ]:
pd.to_datetime(df_all['date'], format='%Y年%m月%d日')

エラー出たので確認しましょう。
どうやら「(1回目)」という文字列が紛れ込んでいるようですね...

In [ ]:
# queryメソッドを使うことで、条件を満たす行を簡単にゲットできる。
# ただし、以下のstr.containsメソッドを使うときに限り、engine='python'とやる必要がある。(その他の操作はいらない)
df_all.query('date.str.contains("回")', engine='python')

In [ ]:
# ピンポイントで修正→(1回目)と(2回目)を削除
df_all['date'] = df_all['date'].map(lambda x : x.split("日")[0])
df_all.query('date.str.contains("回")', engine='python')

# ちなみに、もしこのデータを削除するなら以下です
# df_all = df_all[~df_all["seq"].isin(["第1113回", "第1114回"])] # 条件式の先頭に「~」を入れれば、反対の条件を満たす行を探してくれる。
# df_all = df_all.reset_index(drop=True) # indexが飛び飛びになってしまっているので、リセットする
# df_all

In [ ]:
# 扱いやすくするため、dateを時間型データに直します
df_all['date'] = pd.to_datetime(df_all['date'], format='%Y年%m月%d')
df_all.dtypes

In [ ]:
# 後で分析しやすくするため、dateを年・月・日に分解します
dates = pd.concat([df_all['date'].dt.year, df_all['date'].dt.month, df_all['date'].dt.day], axis=1).set_axis(['year' , 'month', 'day'], axis=1)
df_all = pd.concat([df_all, dates], axis=1)
df_all.head()

In [ ]:
# これも保存しておきましょう
#df_all.to_csv("drive/MyDrive/PiedPiper_Python_個人用/sazae_janken.csv", index=False)

In [ ]:
df_all = pd.read_csv("drive/MyDrive/PiedPiper_Python_個人用/sazae_janken.csv")
df_all.head()

#  特徴量エンジニアリング
予測に用いるデータを作りましょう！  
今回一緒に作っていく特徴量は以下の3つです！

1. 1回前のじゃんけんの手
2. 2回前のじゃんけんの手
3. 新年一回目の放送フラグ

何故この３つなのかというと、先行研究で予測への有効性が示唆されているからです。皆さんもデータ分析する際は**絶対**に先行研究を調べましょう。役に立ちますよ！

In [ ]:
# データを呼び戻す
df_all = pd.read_csv("drive/MyDrive/PiedPiper_Python_個人用/sazae_janken.csv")
df_all.head()

## １・２回前のじゃんけんの手のデータを持つ列

In [ ]:
df_all["hand_pre1"] = df_all["hand"].shift(1, fill_value="パー")
df_all["hand_pre2"] = df_all["hand"].shift(2, fill_value="パー")
df_all.head()

In [ ]:
# 計算に使う用に一行ずつ変換しておく
# これをダミー変数化と言います
a = pd.get_dummies(df_all['hand_pre1'], drop_first=True).set_axis(['チョキ_pre1', 'パー_pre1'], axis=1)
b = pd.get_dummies(df_all['hand_pre2'], drop_first=True).set_axis(['チョキ_pre2', 'パー_pre2'], axis=1)

df_all = pd.concat([df_all, a, b], axis=1)
df_all.head()

yes!　無事に作れましたね！

## 新年一回目の放送フラグ

In [ ]:
# その行が新年一発目かどうかを判断し、そうであれば1、そうでなければ0を保管する列を作れば良い
df_all['is_first'] = 0
date_list = []
date_index = 0

def is_first(row):
  global date_list
  global date_index
  if row['year'] not in date_list:
    #row['is_first'] = 1
    df_all['is_first'][date_index] = 1
    date_list.append(row['year'])
  date_index += 1

df_all.apply(lambda x : is_first(x), axis=1)
df_all['is_first'].value_counts()

# データ分析(EDA)

次に、作った特徴量とサザエさんの出す手に関連性がないか確認します。まあ本当はデータ分析のターンは段階的に行われるものではなくて、前処理→データ分析→(その結果を基に)特徴量エンジニアリングのサイクルを細かく回していき、より良いデータを作っていきます。  
尚、これらの分析をカッコつけて探索的データ分析（Explanatory Data Analysis）と言ったりしますw

とりあえず、今回は先ほど作った3つのデータの検証をしてみましょう。

In [ ]:
# クロス集計したい時はpd.crosstab()を使う
pd.crosstab(df_all['year'], df_all['hand_pre1'])
pd.crosstab(df_all['year'], df_all['hand_pre2'])

In [ ]:
# 1回前のじゃんけんの手
pd.crosstab(df_all['hand'], df_all['hand_pre1'])

上のクロス集計表を見ると、前の手と同じ手を次回に出しづらいことが分かりますよね！  
2回前のじゃんけんの手ではどうなるでしょうか？？

In [ ]:
# 2回前のじゃんけんの手
pd.crosstab(df_all['hand'], df_all['hand_pre2'])

やはりこちらも出しづらいことが分かります！これらは特徴量として加えるべきです！！  
次は新年一回目の放送での出す手を見てみましょう

In [ ]:
# 新年一回目の放送フラグ
pd.crosstab(df_all['hand'], df_all['is_first'])

事前調査通り、年初の放送のジャンケンでは圧倒的にチョキを出しやすいことがデータから分かります。  
これは特徴量に組み込まない手はないですね...！

さて、今まで放置してたdetailに触れていきましょう。detail2に関してはそもそもNaNでないデータが2つしかないため、削除の方向でいいでしょう。しかしdetail1はどうでしょうか？調べてみましょう！

In [ ]:
df_all['is_event'] = df_all['detail1'].notna()
df_all['is_event'] = df_all['is_event'].astype(int)
pd.crosstab(df_all['hand'], df_all['is_event'])

仮説通り、イベントがある時はチョキが出やすそうです！  
これも特徴量に入れましょう

In [ ]:
# ジャンケンの手を計算的に扱いたいので、handカラムのグーチョキパーを0,1,2に変換する
df_all['target'] = df_all['hand'].map(lambda x : 0 if x=='グー' else 1 if x=='チョキ' else 2)
df_all['target'].value_counts()

In [ ]:
# このデータを最終データとして保存しておく
#df_all.to_csv("drive/MyDrive/PiedPiper_Python_個人用/sazae_janken_final.csv", index=False)

# 予測モデル作成
お待たせしました。一番楽しいところです。  
早速AI作り...ではなく、自作AIの実力を評価するための関数を作りましょう。でなければ、本当に強いジャンケン予測器が作れたのかどうか分かりませんからね。

流れとしては、ランダムにジャンケンするモデルを作り、それで評価関数をテストする感じです。

In [ ]:
# データを呼び戻す
df_all = pd.read_csv("drive/MyDrive/PiedPiper_Python_個人用/sazae_janken_final.csv")

In [ ]:
# 最もテキトーなモデル...ランダムモデル！
class random_model:
  def __init__(self):
    self.hands = {0:"グー", 1:"チョキ", 2:"パー"}

  def fit(self, X, y):
    return 

  def predict(self, x):
    return pd.Series([random.randint(0,2) for i in range(len(x))], name='pred')


In [ ]:
# 試しに予測してみる
# (ランダムモデルでは使わないけど)学習データとテストデータを作る
train = df_all.query("year < 2022")
x_train = train[['is_first', 'is_event', 'チョキ_pre1', 'パー_pre1', 'チョキ_pre2', 'パー_pre2']]
y_train = train['target']

test = df_all.query("year >= 2022").reset_index(drop=True)
x_test = test[['is_first', 'is_event', 'チョキ_pre1', 'パー_pre1', 'チョキ_pre2', 'パー_pre2']]
y_test = test['target'].rename('true')

# 予測フェーズ
r_model = random_model()
r_model.fit(x_train, y_test)
pred = r_model.predict(x_test)
pd.concat([pred, y_test], axis=1)

「モデルを評価する」とは、つまり「モデルがどれだけ正確に予測できるかを定量的に表す」ことです。今回はついでなので分類タスクにおける種々の評価指標について実装していきましょう。

今回評価に使う指標は、「正解率」「勝率」です。  
「**正解率**」は私たちが普段使う意味の正解率の考え方に近いです。要は全予測中で正解ラベルと同じだった割合のことですね。

また、今回はジャンケンをするのですから、「**勝率**」も重要かつ気になるでしょう。勝率の定義はサザエさんジャンケン研究所と同じ定義で実装します。つまり、「**引き分けを分母から除いたジャンケンで勝った割合**」ですね。

その他にも分類タスクには色々な種類の評価指標があります。 
参考：https://zero2one.jp/ai-word/accuracy-precision-recall-f-measure/

In [ ]:
from pandas.core.window.doc import window_agg_numba_parameters
# 何回かの過去データを使ってモデルを評価する関数を作る
# 実装対象：「正解率」「適合率」「再現率」「F値」「勝率」
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score, classification_report
from sklearn.model_selection import GridSearchCV

def janken_eval(model, data, columns):
  # print('classification_report = \n', classification_report(y_true=Y_test, y_pred=Y_pred))
  # print('confusion matrix = \n', confusion_matrix(y_true=Y_test, y_pred=Y_pred))
  # print('accuracy = ', accuracy_score(y_true=Y_test, y_pred=Y_pred))
  # print('precision = ', precision_score(y_true=Y_test, y_pred=Y_pred), average='micro')
  # print('recall = ', recall_score(y_true=Y_test, y_pred=Y_pred))
  # print('f1 score = ', f1_score(y_true=Y_test, y_pred=Y_pred))

  # 勝率
  total_win, total_draw, total_lose, total_accuracy = 0, 0, 0, 0
  player_hand_trans = {0:2, 1:0, 2:1}
  d_list = []

  for year in range(1995,2023+1):
    train = data.query(f"year < {year}").reset_index(drop=True)
    test = data.query(f"year == {year}").reset_index(drop=True)
    train_x, train_y = train[columns], train['target']
    test_x, test_y = test[columns], test['target']

    # parameters = [params]
    # clf_cv = GridSearchCV(model(), parameters, n_jobs = -1)
    # clf_cv.fit(train_x, train_y)
    # clf = model(C=clf_cv.best_estimator_.C, random_state=71)
    model.fit(train_x, train_y)
    pred = model.predict(test_x)

    df_y = pd.DataFrame([pred, test_y], index=['pred', 'true']).T
    accuracy = accuracy_score(y_true=df_y['true'], y_pred=df_y['pred'])
    total = len(df_y)
    win = len(df_y[df_y['pred'] == df_y['true']])
    df_y['pred'] = df_y['pred'].map(lambda x : player_hand_trans[x])
    draw = len(df_y[df_y['pred'] == df_y['true']])
    lose = total - win - draw

    d = {}
    d["year"] = year
    d["win"] = win
    d["draw"] = draw
    d["lose"] = lose
    d["winning_percentage"] = f'{(d["win"] / (d["win"]+d["lose"]))*100:.3f}'
    d["accuracy"] = accuracy
    d_list.append(d)

  print()
  df_final = pd.DataFrame(d_list)
  total_win, total_draw, total_lose, total_accuracy =  df_final.sum()[[1,2,3,5]]
  display(pd.DataFrame(d_list))

  print(f'TOTAL：win:  {total_win}, draw:  {total_draw}, lose:  {total_lose}, winning percentage = {(total_win / (total_win+total_lose))*100:.3f}, accuracy = {total_accuracy/(2023+1-1995):.3f}')


In [ ]:
# ランダムモデルで実験
r_model = random_model()

columns = ['is_first', 'is_event', 'チョキ_pre1', 'パー_pre1', 'チョキ_pre2', 'パー_pre2']
janken_eval(r_model, df_all, columns)

ここまでよく頑張りましたね！いよいよ機械学習モデルを作りますよ！  

今回試す機械学習アルゴリズムはロジスティック回帰・サポートベクターマシン(SVM)・ランダムフォレストという手法です。それぞれ良い所はあるのですが、予測能力に限って言えば

ロジスティック回帰 ＜ SVM ＜ ランダムフォレスト

の順に強いです。（＝複雑な予測対象でも対応できる）  
さて、予測モデルの強さによってジャンケンの勝率が変わるのでしょうか！？

In [ ]:
from sklearn.linear_model import LogisticRegression

lr_model = LogisticRegression()

columns = ['is_first', 'is_event', 'チョキ_pre1', 'パー_pre1', 'チョキ_pre2', 'パー_pre2']
janken_eval(lr_model, df_all, columns)

In [ ]:
from sklearn.svm import SVC

svm_model = SVC(C=1)
params = {'C': np.logspace(-2, -1, 10)}

columns = ['is_first', 'is_event', 'チョキ_pre1', 'パー_pre1', 'チョキ_pre2', 'パー_pre2']
janken_eval(svm_model, df_all, columns)

In [ ]:
from sklearn.ensemble import RandomForestClassifier

rf_model = RandomForestClassifier()

columns = ['is_first', 'is_event', 'チョキ_pre1', 'パー_pre1', 'チョキ_pre2', 'パー_pre2']
janken_eval(rf_model, df_all, columns)

In [ ]:
# 最後に使うので、最も性能が良かったモデルを保存する
# モデル保存
#filename = 'drive/MyDrive/PiedPiper_Python_個人用/sazae_janken_model.pickle'
#pickle.dump(lr_model, open(filename, 'wb'))

いかがでしたか？データ分析の力を使えば、サザエさん程度圧倒するのは造作もないのです（？）  
今回はあくまでサザエさんとのジャンケンにおける「お遊びモデル」として実装しました。  
ただ、このモデルを存分に活かす機会ならありますよね？来週のサザエさんとのジャンケンはこのモデルで戦いましょう（？）

データ分析の章に戻って新しい特徴量を見つければ、もっと勝率を上げられるかもしれません。  
ぜひ試してみてください！！👋  

アイデア：
1. その手が出ていない期間（前回出た＝１、前々回出た＝２...）
2. ある年におけるその手が出た割合

# 予測アプリ制作
せっかくサザエさんをボコれるAIを作ったのですから、その実力を試さなくては勿体無いですよねぇ？！  
最後にこのAIを使うインターフェースを整え、アプリケーションにしてしまいましょう！  
そして今週のサザエさんの手を予測し、勝利しましょう！

In [ ]:
# 予測モデル・データを呼び出す
filename = 'drive/MyDrive/PiedPiper_Python_個人用/sazae_janken_model.pickle'
loaded_model = pickle.load(open(filename, 'rb'))
df_all = pd.read_csv("drive/MyDrive/PiedPiper_Python_個人用/sazae_janken_final.csv")

columns = ['is_first', 'is_event', 'チョキ_pre1', 'パー_pre1', 'チョキ_pre2', 'パー_pre2']
janken_eval(loaded_model, df_all, columns)

In [ ]:
# 予測アプリ（関数）を作る
def sazae_janken(model, columns):
  data_for_pred = {} # 辞書型の空データを作る
  trans_hands = {0:"グー", 1:"チョキ", 2:"パー"} # 出力をジャンケンの手に変換する辞書

  for column in columns:
    data_for_pred[column] = input(f"{column}の値は？ → ")

  pon = model.predict(pd.DataFrame(data_for_pred, index=['pred']))
  print(f"来週もまた見てくださいね〜！　ジャン・ケン・ポン！　「{trans_hands[pon[0]]}」　ウフフフフフ❤️")


In [ ]:
df_all.head()

In [ ]:
# 勝負
columns = ['is_first', 'is_event', 'チョキ_pre1', 'パー_pre1', 'チョキ_pre2', 'パー_pre2']
sazae_janken(loaded_model, columns)

# 参考文献
- サザエさんじゃんけん研究所 公式ウェブサイト、サザエさんじゃんけん研究所、http://park11.wakwak.com/~hkn/
- サザエさんじゃんけん白書、サザエさんじゃんけん研究所、http://park11.wakwak.com/~hkn/report.pdf
- Sazae_R、八寿、https://github.com/yaju/Sazae_R
- https://w.atwiki.jp/sazaesannokiroku/pages/21.html


# オマケ
本colab製作者の試行錯誤履歴

In [ ]:
# 早速作っていこう
df_all['hand_pre3'] = df_all['hand'].shift(3, fill_value="パー")
c = pd.get_dummies(df_all['hand_pre3'], drop_first=True).set_axis(['チョキ_pre3', 'パー_pre3'], axis=1)

df_all = pd.concat([df_all, c], axis=1)
df_all.head()

In [ ]:
lr_model = LogisticRegression()

columns = ['is_first', 'is_event', 'チョキ_pre1', 'パー_pre1', 'チョキ_pre2', 'パー_pre2', 'チョキ_pre3', 'パー_pre3']
janken_eval(lr_model, df_all, columns)

In [ ]:
df_all['is_first_pre'] = df_all['is_first'].shift(1, fill_value=0)
df_all['is_event_pre'] = df_all['is_event'].shift(1, fill_value=0)

df_all.head()

In [ ]:
lr_model = LogisticRegression()

columns = ['is_first', 'is_event', 'is_first_pre', 'is_event_pre', 'チョキ_pre1', 'パー_pre1', 'チョキ_pre2', 'パー_pre2', 'チョキ_pre3', 'パー_pre3']
janken_eval(lr_model, df_all, columns)

In [ ]:
f = pd.get_dummies(df_all['month'], drop_first=True).set_axis(['2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12'], axis=1)

df_all = pd.concat([df_all, f], axis=1)
df_all.head()

In [ ]:
lr_model = LogisticRegression()

columns = ['is_first', 'is_event', 'is_first_pre', 'is_event_pre', 'チョキ_pre1', 'パー_pre1', 'チョキ_pre2', 'パー_pre2', 'チョキ_pre3', 'パー_pre3',
                '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12']
janken_eval(lr_model, df_all, columns)

In [ ]:
# 次回予告の役
# 波平→フネ→マスオ→カツオ→ワカメ→タラちゃんの順
nexters = {0:"波平", 1:"フネ", 2:"マスオ", 3:"カツオ", 4:"ワカメ", 5:"タラちゃん"}

In [ ]:
import numpy as np
import pandas as pd

# サンプルデータの生成
np.random.seed(0)
rounds = np.arange(1, 11)
hands = np.random.choice(['guu', 'choki', 'pa'], size=10)
df = pd.DataFrame({'round': rounds, 'hand': hands})

# 各手のカウントを初期化
counts = {'guu': 0, 'choki': 0, 'pa': 0}

# 各手のカウントを更新する関数
def update_counts(row):
    global counts
    counts = {k: counts[k]+1 for k in counts} # 1を足す
    counts[row['hand']] = 0 # 出した手のカウントを初期化
    row['guu'] = counts['guu']
    row['choki'] = counts['choki']
    row['pa'] = counts['pa']
    # if counts[row['hand']] == 1:
    #     counts = {k: counts[k]+1 for k in counts}
    return row

# applyメソッドで各行に対して更新を実行
df[['guu', 'choki', 'pa']] = df.apply(update_counts, axis=1)[['guu', 'choki', 'pa']]

print(df)


In [ ]:
# import random

# # 完成したい文字列
# target_str = "ドドスコスコスコ"
# rand_str = ""

# # ランダムに文字列を出力しつづける
# while True:
#     rand_str_tmp = random.choice(["ドド", "スコ"])
#     rand_str += rand_str_tmp
#     print(rand_str, end="")
    
#     # target_strと一致しているかどうかを確認する
#     if len(target_str) <= len(rand_str):
        
#         # target_strが空になった場合、プログラムを終了する
#         if rand_str.endswith(target_str):
#             print("ラブ注入❤️")
#             break


# LICENCE

このcolabは[githubのREADME](https://github.com/agu-piedpiper/piedpiper-python)にて記載の通りのライセンスに従います。